# Spider Workshop - Exercise 6: A Full Spider!
(These exercises were prepared for ganja.js by Steven De Keninck for [GAME23](https://bivector.net/game2023.html).)

At this point it becomes more of a programming exercise. The idea is to duplicate
our spider leg 8 times, and animate the entire spider by setting new 'goals' for
each 'foot', updating targets when goals are far enough away, and making sure not
to ever do this for adjacent legs simultaniously.

In [1]:
%pip install -q kingdon anywidget==0.9.9 ipywidgets==8.1.3

Note: you may need to restart the kernel to use updated packages.


Create a geometric algebra with 3 positive and one null basis vector:

In [2]:
from kingdon import Algebra, MultiVector

alg = Algebra(3, 0, 1)

In lesson 4 we learned how to translate along a line. Let's make this into a function:

In [3]:
def tr(line, dist: float):
    """ Translate a distance along a line. """
    horizon = alg.blades.e0
    origin = horizon.dual()
    return ( (-0.5 * dist * horizon) ^ (line.normalized() | origin)).exp()

We create a function that produces a spiderleg for us, at a given angle around the e13 axis:

In [4]:
from dataclasses import dataclass

@dataclass
class Leg:
    chain: list[MultiVector]
    start: float = 0
    active: bool = False

def create_leg(angle=0) -> Leg:
    R = (-0.5 * angle * alg.blades.e13).exp()
    chain = [
        alg.vector(e0=1, e1=0, e2=0, e3=0).dual(),
        alg.vector(e0=1, e1=0.5, e2=0, e3=0).dual(),
        alg.vector(e0=1, e1=1, e2=0, e3=0).dual(),
        alg.vector(e0=1, e1=1.5, e2=0, e3=0).dual(),
    ]
    chain = [R >> p for p in chain]
    return Leg(chain)

IK function from the previous exercise with one extra constraint: to keep the last segment of the leg straight.

In [5]:
UP = alg.blades.e2.dual()

def inverse_kinematics(C, Base, Target) -> None:
    """Our IK function takes a chain, a base and a target, and updates the chain *in place* to a new chain."""
    C[3] = Target
    # Constraint 3: keep the last segment straight.
    C[2] = C[3] + 0.5 * UP
    
    for i in reversed(range(3)): 
        C[i] = tr(C[i] & C[i+1], 0.5) >> C[i+1]
    
    # Constraint 1: fix the leg plane
    plane = Base & Target & UP
    for i in range(1, len(C) - 1):
        C[i] = (C[i] | plane) / plane
    
    # Constraint 2: keep the first segment up
    plane2 = alg.blades.e13 | Base
    signed_distance = (plane2 & C[1]).e
    if signed_distance < 0:
        C[1] = (C[1] | plane2) / plane2
    
    # angle = (((C[1] & C[2] | C[2]) ^ plane) | (C[2] & C[3])).e
    # if angle < 0:
    #     C[2] =  (C[1] & C[3]).normalized() >> C[2]
    
    C[0] = Base
    for i in range(1, 3 + 1):
        C[i] = tr(C[i-1] & C[i], -0.5) >> C[i-1]

Now we create a base, some targets, and some legs.

In [6]:
import math

base = alg.vector(e0=1, e1=-0.5, e2=0, e3=0).dual()
targets = []
legs = []
for i in range(10):
    if i == 0 or i == 5: 
        continue
    angle = i*2*math.pi/10 + math.pi/2
    target = alg.vector(e0=1, e1=0.8, e2=-1, e3=0).dual()
    targets.append( (-0.5 * angle * alg.blades.e13).exp() >> target)
    legs.append(create_leg(angle))

Now lets create the future positions for our targets. So targets is where the legs are, goal where they want to be. 

In [7]:
INITIAL_GOALS = [t + 0 for t in targets]  # copy the targets.

Render these elements using the `Algebra.graph` function. If you provide a function without arguments to `Algebra.graph`, the function will be re-evaluated everytime you drag a point.
So pro-tip for the coming exercises: define everything within `graph_func`. 

In [8]:
import itertools

time = 0

def graph_func():
    global time, base

    # Grab the time
    time += 1/60

    # Resolve the IK
    for leg, target in zip(legs, targets):
        inverse_kinematics(leg.chain, base, target)
    # move the goals forward

    # loop over all legs. If our target is too far from our goal, and the legs before
    # and after us are not active, then we become active.

    # loop over all legs, if we are active move our target towards its goal.

    # Update our base to be at the average of our targets.

    # now return a list of things to render.
    return [
        "Spider Workshop - Spider!",
        *itertools.chain(*[itertools.pairwise(leg.chain) for leg in legs]),  # Blessed are those that know the standard library.
        base, "B"
    ]

alg.graph(
    graph_func,
    grid=True, labels=True, lineWidth=4, pointRadius=4, animate=True,
)

GraphWidget(cayley=[['1', 'e0', 'e1', 'e2', 'e3', 'e01', 'e02', 'e03', 'e12', 'e13', 'e23', 'e012', 'e013', 'e…

## Exercises:

Things are now down to programming mechanics. Follow the guides in the code
and try to get the spider to walk around.

1. To move the goals forward, you could rotate them a little bit each step
   using a rotor. Once you calculate the rotor R, you can move the goals with
     `goals = [R >> g for g in goals]`

2. We now need to check if our goals have moved enough for our spider to move
   its leg to the new goal. You can use the following check
     `(targets[i] & goals[i]).norm().e > 0.15`

   It creates the line between the target and the goal, and checks its length.
   We also only want to become active if our leg, the leg in front of us, and 
   the leg behind us are not active. 
   If all these conditions are met, we become an active leg. In this case you
   will want to record the start time of the animation and take a copy of
   the current target position. 

   `legs[i].start = time; legs[i].active = true; legs[i].pos = 1*targets[i];`

3. Loop over all legs again, this time to update the active legs and animate
   their target position. You want targets[i] to animate from legs[i].pos to
   goals[i], and lift a bit while doing so. If the goal is reached, the leg
   should also become inactive.

4. We will also need to update the position of our base each frame to the 
   average position of all current targets (moved slightly up). To do this 
   use the built-in `sum` function. 

   sum(targets).normalized()+0.35*alg.blades.e2.dual()  

## Bonus

5. Add shadows.

In [9]:
from exercises.solution_widget import SolutionWidget

SolutionWidget(exercise='spider6')

SolutionWidget()